# API access to napari-convpaint

This notebooks demonstrates how to run the software behind the napari-convpaint plugin completely independently from napari itself.


## Imports

In [30]:
%load_ext autoreload
%autoreload 2

# import napari and its screenshot function
import napari
from napari.utils.notebook_display import nbscreenshot

# import what we need from conv_paint
from napari_convpaint.conv_paint import ConvPaintWidget
from napari_convpaint.conv_paint_utils import Hookmodel
from napari_convpaint.convpaint_sample import create_annotation_cell3d
from napari_convpaint.conv_paint_utils import (filter_image_multioutputs, get_features_current_layers,
get_multiscale_features, train_classifier, predict_image)
from napari_convpaint.conv_paint_utils import extract_annotated_pixels
 
# import the other general modules used
import numpy as np
import skimage
import tifffile

# import pytorch
import torch
from torchvision.transforms import Compose, Resize, ToTensor, Normalize

# import pillow.image
from PIL import Image



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data

First we load an image and the corresponding annotation. We simply display the result as a napari screenshot.

In [54]:
# Load 3D image with 2 channels (cell borders and nuclei)
image_original = skimage.data.cells3d()
# Take layer in middle of cell (30 of 0-59) and take 2nd channel (nuclei)
image = image_original[30,1]
# Load annotation defined in conv_paint
labels = create_annotation_cell3d()[0][0]

# Take crops of image and annotation
image = image[60:188, 0:128]
labels = labels[60:188, 0:128]
original_im_shape = image.shape
original_im_shape

218

In [32]:
# create a napari viewer
viewer = napari.Viewer()
# add the loaded image to it
viewer.add_image(image)
# add the loaded labels/annotation
viewer.add_labels(labels)

<Labels layer 'labels' at 0x21e1af80f40>

This are the data we are working with:

In [33]:
# show a screenshot of the napari viewer here in the notebook
# nbscreenshot(viewer)

In [5]:
#tifffile.imwrite('label_cell3d.tiff', viewer.layers['Labels'].data)

## Create model

The Hookmodel is a wrapper around PyTorch models. It allows to list all modules (layer) of a model and to place "hooks" in order to get the output of chosen layers.

In [36]:
# model = Hookmodel(model_name='vgg16')

dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
# dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')
# dinov2_vitl14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
# dinov2_vitg14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14')

# Choose the model to use
model = dinov2_vits14

Using cache found in C:\Users\roman/.cache\torch\hub\facebookresearch_dinov2_main


If running on a CUDA GPU, it is possible to use ```model = Hookmodel(model_name='vgg16', use_cuda=True)``` to run inference on the GPU.

Here is the complete list of modules of the model:

From that list we can pick specific layers that will work as feature extractors:

In [39]:
# Convert to pillow image
image_pillow = Image.fromarray(image)
image_pillow = image_pillow.convert("RGB")

# Preprocess image
preprocess = Compose([
    Resize((224, 224)),  # Resize to the input size expected by the model
    ToTensor(),  # Convert to PyTorch tensor
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Normalize to ImageNet mean and std
])
image_pre = preprocess(image_pillow)

Then we add hooks at these layers, i.e. we capture their output. The flow through the network is interrupted at the last selected layer:

## Feature extraction

Now that the model is defined, we can run an image through it and recover the outputs at the "hooked" layers. All this is wrapped insisde the ```get_features_current_layers``` function.

There are several options to run the feature extraction:
- with ```use_min_features=True```, only the n-first features of each output layer are selected, n being the number of features of the layer which outputs the least of them
- ```scalings``` specifies the levels of downscaling to use (1 is the original size)
- ```order``` specifies the spline order used to upscale small feature maps

In [47]:
# Add an extra batch dimension and move image to the GPU if available
images = image_pre.unsqueeze(0)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
images = images.to(device)
model = model.to(device)

# Pass image through the model (assuming images is a batch of test images)
with torch.no_grad():
    # features=model.forward_features(torch.nn.functional.interpolate(images,(448,448)))['x_norm_patchtokens']
    features=model.forward_features(images)['x_norm_patchtokens']

features = features.permute(0,2,1)
features = features.reshape(1,384,16,16)#.squeeze(0)

# features_int = torch.nn.functional.interpolate(features, scale_factor=14)
features_int = torch.nn.functional.interpolate(features, size=original_im_shape)
features_np = features_int.numpy()
features_np = np.squeeze(features_np, axis=0)

In [50]:
features, targets = extract_annotated_pixels(features_np, labels)
print(targets.shape)
print(features.shape)

(222,)
(222, 384)


The selected output layers have 64 and 256 output features, for a total of 320 features. This matches the number of features of the ```features``` dataframe as shown above.

## Train and use Classifier
Finally we can train a classifier:

In [51]:
random_forest = train_classifier(features, targets)

And do a prediction. Note that the same settings as those used for training need to be used here for ```scalings```, ```order``` and ```use_min_features```:

In [52]:
predicted = predict_image(image, model, random_forest, scalings=[1,2], order=1, use_min_features=False)

TypeError: 'method' object is not subscriptable

## Visualize Results
And finally we can visualize the output (and quantify its quality):

In [15]:
viewer.add_labels(predicted)

<Labels layer 'predicted' at 0x2d707268d30>

In [ ]:
# nbscreenshot(viewer)

## Tests Roman

In [ ]:
# import matplotlib.pyplot as plt

# plt.imshow(labels)

np.sum(labels > 0)

# plt.imshow(features)


In [4]:
# CREATE AND SHOW FULL OUTPUT OF A LAYER OF VGG16 (= 64 FEATURES)
def get_layer_features(image, layer, show_napari = False, interpolate = False):
        
    model = Hookmodel(model_name='vgg16')


    all_layers = [key for key in model.module_dict.keys()]
    # Choose just 1 layer, and register a hook there
    if isinstance(layer, str):
        layers = [layer]
    elif isinstance(layer, int):
        layers = [all_layers[layer]]
    
    # layers = ['features.30 MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False) avgpool AdaptiveAvgPool2d(output_size=(7, 7))']
    model.register_hooks(selected_layers=layers)

    # Get features using only this first layer and without scaling
    features, targets = get_features_current_layers(
        model=model, image=image, annotations=image, scalings=[1], use_min_features=False, order=interpolate)

    # Convert the DataFrame to a numpy array
    features_array = features.values
    # Get the shape of the image
    image_shape = image.shape
    # Reshape the features array to match the image shape and add the second dimension of features as the third dimension
    features_image = features_array.reshape(*image_shape, -1)

    # Move the last dimension to the first position
    features_image = np.moveaxis(features_image, -1, 0)
    # print(features.shape)
    # print(features_image.shape)

    # Now you can view the new_features using napari
    if show_napari: napari.view_image(features_image)
    return features_image

In [5]:
# RUN

# image = image.T

# Get features of multiple (all) layers
conv_layers = [0,2]#,5,7,10,12,14,17,19,21,24,26,28]
all_conv = [get_layer_features(image, l) for l in conv_layers]


### Pad first dimension of the layers with fewer features and concatenate all layers into a 4D Image

# Get the shapes of all outputs
shapes = [output.shape for output in all_conv]
# Find the maximum shape in each dimension
max_shape = np.max(shapes, axis=0)
# Pad all outputs to have the max shape
from numpy.lib import pad
all_conv_padded = np.array([pad(output, [(0, max_dim - dim) for dim, max_dim in zip(output.shape, max_shape)]) for output in all_conv])

# Show in Napari
napari.view_image(all_conv_padded)

Viewer(camera=Camera(center=(0.0, 63.5, 63.5), zoom=3.350573501872659, angles=(0.0, 0.0, 90.0), perspective=0.0, mouse_pan=True, mouse_zoom=True), cursor=Cursor(position=(0.0, 31.0, 0.0, 0.0), scaled=True, size=1, style=<CursorStyle.STANDARD: 'standard'>), dims=Dims(ndim=4, ndisplay=2, last_used=0, range=((0.0, 2.0, 1.0), (0.0, 64.0, 1.0), (0.0, 128.0, 1.0), (0.0, 128.0, 1.0)), current_step=(0, 31, 63, 63), order=(0, 1, 2, 3), axis_labels=('0', '1', '2', '3')), grid=GridCanvas(stride=1, shape=(-1, -1), enabled=False), layers=[<Image layer 'all_conv_padded' at 0x1b05f5a8130>], help='use <2> for transform', status='Ready', tooltip=Tooltip(visible=False, text=''), theme='dark', title='napari', mouse_over_canvas=False, mouse_move_callbacks=[], mouse_drag_callbacks=[], mouse_double_click_callbacks=[], mouse_wheel_callbacks=[<function dims_scroll at 0x000001B050F6B280>], _persisted_mouse_event={}, _mouse_drag_gen={}, _mouse_wheel_gen={}, keymap={})

In [7]:
# SHOW VGG16 MODEL SUMMARY IN PYTORCH

import torch
import torchvision.models as models

# Load vgg16 model
vgg16 = models.vgg16()

# Print model summary
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1